# Objekterkennung auf dem Boneage Datensatz

Bei der Objekterkennung geht es darum, Objekte innerhalb eines Bildes zu finden und zu klassifizieren. Anders als bei der Segmentierung werden die Objekte dabei üblicherweise mit Bounding-Boxen umgeben, die alle Pixel des Objekts enthalten sollen, darüber hinaus aber auch Teile des Hintergrunds oder umgebender Objekte enthalten.

<img src="https://lh6.googleusercontent.com/TLLJ9h4eOKM8loxc8SpKaVPaX0laf_dtsGr7BIWhf36PErnKFweZTGDnhKYsVJ68g9AAiEzyQ67iydfSvVg2TwoKvJxbmcutpPvWmW7yFB7v9CInUtX5r4QtelyaHJCqhGsgzysC" />
Quelle: https://www.clarifai.com/blog/classification-vs-detection-vs-segmentation-models-the-differences-between-them-and-how-each-impact-your-results


In diesem Notebook word ein Modell zur Objekterkennung für den Boneage Datensatz mit [M-RCNN](https://github.com/matterport/Mask_RCNN) (welches wiederum Keras und Tensorflow als Backend nutzt) trainiert. In diesem Modell werden zwei getrennte Netze trainiert: eins für die grobe Idenfitikation von relevanten ROIs (Region Proposal Network, RPN) und ein weiteres zur Klassifikation bzw. genaueren Einteilung dieser ROIs.

Der Code ist adaptiert von [train_shapes](https://github.com/matterport/Mask_RCNN/blob/master/samples/shapes/train_shapes.ipynb).

## Datensatz

Der [Boneage](http://rsnachallenges.cloudapp.net/competitions/4) Datensatz besteht aus 12.800 Hand-Röntgenaufnahmen von Kindern und Jugendlichen, die sich in 12.600 Trainings- und 200 Testbilder aufteilen. Ziel der RSNA-Challenge ist die Altersbestimmung des zugehörigen Patienten aufgrund dieser Röntgenbilder.

Der hier verwendete Datensatz verwendet ein [Subset](https://doi.org/10.1371/journal.pone.0207496) ([GitHub](https://github.com/razorx89/rsna-boneage-ossification-roi-detection) mit 240 Trainings- und 89 Validierungsbildern für die _Regions Of Interest_ (ROI) von für die Altersbestimmung relevanter Gelenke:

- distal interphalangeal joints (DIP, unten in grün)
- proximal interphalangeal joints (PIP, unten in türkis)
- metacarpophalangeal joints (MCP, unten in blass grün)
- Handgelenk
- Elle
- Speiche

<img src="https://raw.githubusercontent.com/razorx89/rsna-boneage-ossification-roi-detection/master/example.png"  width="600" />

Für weitere Informationen sei auf das Paper [Ossification area localization in pediatric hand radiographs using deep neural networks for object detection](https://doi.org/10.1371/journal.pone.0207496) von S. Koitka, A. Demircioglu, M.S. Kim, C.M. Friedrich, F. Nensa verwiesen.

Das Ziel unseres Modells ist die Detektion dieser Gelenke und die automatische Bestimmung von entsprechenden ROIs.

# 1. Imports und Setup

Wie bei der Klassifikation werden hier benötigte Bibliotheken importiert, Funktionen definiert und Konstanten festgelegt.

In [ ]:
import os
import logging
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import csv
import imgaug
from scipy.interpolate import make_interp_spline, BSpline


# Directories
MRCNN_DIR = os.path.abspath("/mrcnn")
ROOT_DIR = os.path.abspath("/workspace")
PRETRAINED_MODEL_DIR = os.path.abspath("/models")
DATA_DIR = os.path.abspath("/data")

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "models")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(PRETRAINED_MODEL_DIR, "mask_rcnn_coco.h5")

import tensorflow as tf

# Tell TF to not use all GPU RAM
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=6*1024)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
        print(e)
print("Tensorflow setup done")

from tensorflow.keras.callbacks import Callback


import asyncio
import concurrent
asyncio.get_event_loop().set_default_executor(concurrent.futures.ThreadPoolExecutor(max_workers=6))

# Import Mask RCNN
sys.path.append(MRCNN_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

%matplotlib inline 

# Functions

def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

def filter_result(r, num_elements_per_class_id):
    keep_indices = set()
    
    ids_to_scores = {}
    class_ids = set(r['class_ids'])
    
    class_counts = np.zeros(len(class_ids)+10)
    
    for i in range(len(r['class_ids'])):
        class_id = r['class_ids'][i]
                
        if class_counts[class_id] < num_elements_per_class_id[class_id]:
            class_counts[class_id] += 1
            keep_indices.add(i)
    
    new_r = { 'rois': np.array([]), 'masks': np.array([]), 'class_ids': np.array([]), 'scores': np.array([])}
    for index in keep_indices:
        for x in ['rois', 'class_ids', 'scores']:
            new_r[x] = np.append(new_r[x], r[x][index])
    
    # masks
    delete_indices = list(set(range(len(r['class_ids']))).symmetric_difference(keep_indices))
    new_r['masks'] = np.delete(r['masks'], delete_indices, axis=2)
    
    new_r['rois'] = new_r['rois'].reshape(len(keep_indices),4)
    new_r['class_ids'] = new_r['class_ids'].astype(int)
    
    return new_r

def smooth(values, num_points = 100):
    xnew = np.linspace(0,len(values)-1,num_points) #100 represents number of points to make between T.min and T.max
    spl = make_interp_spline(list(range(len(values))), values, k=3) #BSpline object
    values_smooth = spl(xnew)
    
    return xnew, values_smooth

class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.loss = []
        self.rpn_class_loss = []
        self.rpn_bbox_loss = []
        self.mrcnn_class_loss = []
        self.mrcnn_bbox_loss = []

    def on_batch_end(self, batch, logs={}):
        self.loss.append(logs.get('loss'))
        self.rpn_class_loss.append(logs.get('rpn_class_loss'))
        self.rpn_bbox_loss.append(logs.get('rpn_bbox_loss'))
        self.mrcnn_class_loss.append(logs.get('mrcnn_class_loss'))
        self.mrcnn_bbox_loss.append(logs.get('mrcnn_bbox_loss'))

print("Imports done")
print("Directories:")
print("Root directory:", ROOT_DIR)
print("Model directory:", MODEL_DIR)
print("Pre-trained model directory:", PRETRAINED_MODEL_DIR)
print("M-RCNN directory:", MRCNN_DIR)
print("Datasets directory:", DATA_DIR)

## 1.1 Ordnerstruktur

Hier kann mit einfachen Linux-Kommandozeilenbefehlen die Ordnerstruktur untersucht werden

In [ ]:
!ls -l /data
!ls -l /data/boneage
!ls -l /workspace

# 2. Konfiguration

An dieser Stelle wird die Konfiguration des Modells festgelegt. Zur Optimierung werden wir später an diesen Punkt zurück kehren.

Die wichtigsten Stellschrauben sind:

- `IMAGES_PER_GPU`: dies ist in unserem Fall die Batchgröße, da `GPU_COUNT` immer 1 bleibt. Diese Zahl kann erhöht werden, da wir aber nur 40% des GPU-Speichers zur Verfügung haben, kann dies zur Trainingsabstürzen führen.
- `RPN_ANCHOR_SCALES`: Ein "Anchor" ist bei dem MRCNN-Modell eine vom ersten Netzwerk (RPN) vorgeschlagene Region, hier kann festgelegt werden, welche Größenordnungen dieser Anchor haben dürfen (bezogen auf das skalierte Bild, nicht das Originalbild). Da wir nach kleinen (DIP/PIP/MCP) und mittelgroßen (Wrist) Objekten suchen, könnte hier z.B. zur Optimierung die 64 entfernt werden.
- `TRAIN_ROIS_PER_IMAGE`: Wie viele ROIs sollen pro Bild gesucht werden? Für kurze Trainingsläufe ist es besser hier eine kleine Zahl zu wählen, da ansonsten zu viele Objekte auf den Bildern gefunden werden. Für längere Trainingsläufe sollte die Zahl etwa dem __Dreifachen der erwarteten Objekte__ entsprechen, da der Klassifikator dann die false positives noch herausfiltern kann.
- `LEARNING_RATE`: die wohl wichtigste Stellschraube während des Trainingsprozesses. Ist diese Zahl zu klein, dann wird nur langsam gelernt, ist diese Zahl zu groß dann besteht die Gefahr dass das Netzwerk nicht konvergiert, der Netzwerkfehler also über die Zeit konstant bleibt oder fluktuiert. Sinnvolle Werte liegen zwischen 0.001 und 0.00001
- `LEARNING_MOMENTUM`: beeinflusst den Einflusst des letzten Gewichtsupdates auf das aktuelle Gewichtsupdate. Kann z.B. auf 0.5 reduziert werden oder deaktiviert werden.
- `WEIGHT_DECAY`: beeinflusst das Schrumpfen von Gewichten die nicht verändert wurden. Kann deaktiviert werden, sollte nicht zu viel erhöht werden.
- `DETECTION_MIN_CONFIDENCE`: gibt an ab welchen Konfidenzlevel das Modell ein gefundenes Objekt ausgeben soll. Wenn zu wenige Objekte angezeigt kann diese Zahl reduziert werden. Dies kann jedoch dazu führen, dass öfter falsche Objekte angezeigt werden.
- `STEPS_PER_EPOCH`: Aus wie vielen Schritten (also Mini-Batches) eine Epoche besteht. Da wir 238 Bilder und eine Batchgröße von 8 haben, brauchen wir `238 / 8 = 29.75`, also 30 Schritte um einmal über alle Bilder zu iterieren.
- `VALIDATION_STEPS`: Nach jeder Epoch wird der Validierungsfehler berechnet, wir haben 89 Validierungsbilder, also brauchen wir 11 Schritte um über diese zu iterieren. Um die Trainingszeit zu reduzieren, wird hier nur die Hälfte genommen – dieser Wert hat keinen direkten Einfluss auf das Training, gibt uns nur eine grobe Orientierung wie gut unser Modell generalisiert.

Aufgaben:

- Wählen Sie eine geeignete Lernrate (`LEARNING_RATE`)

In [ ]:
class BoneAgeConfig(Config):
    """Configuration for training on the bone age dataset.
    Derives from the base Config class and overrides values specific
    to the bone age dataset.
    """
    # Give the configuration a recognizable name
    NAME = "bone_age"

    # Train on 1 GPU and 8 images per GPU. We can put multiple images on each
    # GPU because the images are small. Batch size is 8 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 8

    # Number of classes (including background)
    #NUM_CLASSES = 1 + 5 + 4 + 5 + 3  # background + 5xPIP + 4xDIP + 5xMCP + Wrist + Ulna + Radius
    NUM_CLASSES = 1 + 6  # background + PIP + DIP + MCP + Wrist + Ulna + Radius

    # Use small images for faster training. Set the limits of the small side
    # the large side, and that determines the image shape.
    IMAGE_MIN_DIM = 128
    IMAGE_MAX_DIM = 128

    # Use smaller anchors because our image and objects are small
    #RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)  # anchor side in pixels
    RPN_ANCHOR_SCALES = (8, 16, 32, 64)  # anchor side in pixels

    # Reduce training ROIs per image because the images are small and have
    # few objects. Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 50
    
    LEARNING_RATE = ___LERNRATE_EINTRAGEN___
    
    LEARNING_MOMENTUM = 0.9
    
    WEIGHT_DECAY = 0.0001
    
    MINI_MASK_SHAPE=(128,128)
    
    DETECTION_MIN_CONFIDENCE = 0.75

    # Use a small epoch since the data is simple
    STEPS_PER_EPOCH = 30

    # use small validation steps since the epoch is small
    VALIDATION_STEPS = 5
    
config = BoneAgeConfig()
config.display()

# 3. Datensatz

Hier wird die Datensatz-Klasse definiert, in der u.a. die folgenden Methoden vorhanden sind:

* load_bone_age(): Initialisiert den Datensatz, indem Klassen und Bilder aus der CSV geladen werden
* load_image(): Lädt ein einzelnes Bild aus dem Datensatz – diese Funktion wird von der Klasse [`utils.Dataset`](https://github.com/matterport/Mask_RCNN/blob/master/mrcnn/utils.py#L239) geerbt, die aufgrund des in `load_bone_age()` für das Bild übergebenen Pfades das Bild vom Dateisystem lädt.
* load_mask(): Lädt die Gelenk-ROIs

In [ ]:
class BoneAgeDataset(utils.Dataset):
    """Generates the bone age dataset.
    """

    def load_bone_age(self, dataset_dir, annotation_file):
        """Load the requested subset of the bone age dataset.
        """
        # Add classes
        self.add_class("bone_age", 1, "PIP")
        self.add_class("bone_age", 2, "DIP")
        self.add_class("bone_age", 3, "MCP")
        self.add_class("bone_age", 4, "Wrist")
        self.add_class("bone_age", 5, "Ulna")
        self.add_class("bone_age", 6, "Radius")

        # Add images from CSV
        with open(annotation_file, 'r') as input_file:

            reader = csv.reader(input_file, delimiter=',')

            # Skip header
            header = next(reader)

            image_dir = dataset_dir
            current_filename, current_width, current_height = [None, None, None]
            
            annotations = []
            i = 1
            for row in reader:
                filename, width, height, class_name, xmin, ymin, xmax, ymax = row
                
                width, height, xmin, ymin, xmax, ymax = [int(width), int(height), int(xmin), int(ymin), int(xmax), int(ymax)]
                
                
                if filename != current_filename and current_filename != None:
                    #annotations.sort(key=lambda x: x[0] + str(x[1]))
                    #annotations = list(map(incremental_class_names, annotations))
                    
                    self.add_image(
                        "bone_age", image_id=i,
                        path=os.path.join(image_dir, current_filename),
                        width=current_width,
                        height=current_height,
                        annotations=annotations)
                    i += 1
                    annotations = []
                    #for c in class_name_counters:
                        #class_name_counters[c] = 0
                
                current_filename = filename
                current_width = width
                current_height = height
                annotations.append([class_name, xmin, ymin, xmax, ymax])
                

    def image_reference(self, image_id):
        """Return the shapes data of the image."""
        info = self.image_info[image_id]
        if info["source"] == "bone_age":
            return "https://github.com/razorx89/rsna-boneage-ossification-roi-detection"
        else:
            super(self.__class__).image_reference(self, image_id)

    def load_mask(self, image_id):
        """Generate instance masks for instances in the given image.
        """
        instance_masks = []
        class_ids = []
        image_info = self.image_info[image_id]
        annotations = image_info["annotations"]
        # Build mask of shape [height, width, instance_count] and list
        # of class IDs that correspond to each channel of the mask.
        for annotation in annotations:
            class_id = self.class_names.index(annotation[0])
            if class_id:
                m = np.zeros((image_info["height"], image_info["width"]))
                m[annotation[2]:annotation[4], annotation[1]:annotation[3]] = 1.
                
                # Some objects are so small that they're less than 1 pixel area
                # and end up rounded out. Skip those objects.
                if m.max() < 1:
                    continue
                
                instance_masks.append(m)
                class_ids.append(class_id)

        # Pack instance masks into an array
        masks = np.reshape(np.stack(instance_masks, axis=2).astype(np.bool), (1, image_info["height"], image_info["width"], len(class_ids)))[0]
        class_ids = np.array(class_ids, dtype=np.int32)
        return masks, class_ids

Hier werden nun Trainings- und Validierungsdatensatz initialisiert. Die Bilder liegen gesammelt im Verzeichnis `/data/boneage/train`, in den zugehörigen CSV-Dateien sind die oben erwähnten Subsets für Training und Validierung mit den zugehörigen ROIs definiert.

Fehlermeldungen der Art `"write error: broken pipe"` können ignoriert werden

In [ ]:
files = !ls -l /data/boneage/train | wc -l
print ("Files in bonage directory:", files[0])

lines = !cat /data/boneage/train-bboxes.csv | wc -l
print("Files in training dataset:", int(lines[0]) // 17)

lines = !cat /data/boneage/validation-bboxes.csv | wc -l
print("Files in validation dataset:", int(lines[0]) // 17)
!cat /data/boneage/train-bboxes.csv | head -n 10

Aufgaben:

- Tragen Sie die passenden Pfade für die CSV-Dateien ein

In [ ]:
# Training dataset
dataset_train = BoneAgeDataset()
dataset_train.load_bone_age("/data/boneage/train",
                            "___CSV-PFAD_EINTRAGEN___")
dataset_train.prepare()

color_palette = visualize.random_colors(len(dataset_train.class_names))

# Validation dataset
# training and validation data are subsets from the same directory
dataset_val = BoneAgeDataset()
dataset_val.load_bone_age("/data/boneage/train",
                        "___CSV-PFAD_EINTRAGEN___")
dataset_val.prepare()

## 3.1 Bilder anzeigen

Hier werden nun zufällig ein paar Bilder mit den zugehörigen ROIs geladen und angezeigt. Diese Zelle kann mehrfach ausgeführt werden. Falls die Zelle anfängt zu scrollen kann dies mit `Shift + o` deaktiviert werden.

In [ ]:
# change this to show more or fewer images
NUM_IMAGES = 2

# Load and display random samples
image_ids = np.random.choice(dataset_train.image_ids, NUM_IMAGES, replace=False)
for image_id in image_ids:
    image = dataset_train.load_image(image_id)
    masks, class_ids = dataset_train.load_mask(image_id)
    bbox_list = list(map(lambda x: [x[2], x[1], x[4], x[3]], dataset_train.image_info[image_id]["annotations"]))
    bbox = np.array(bbox_list)
    print("Image shape: " + str(image.shape))
    print("Mask shape: " + str(masks.shape))
    print("Bbox shape: " + str(bbox.shape))
    colors = [color_palette[id] for id in class_ids]
    visualize.display_instances(image, bbox, masks, class_ids,
                            dataset_train.class_names, show_mask=False, show_polygon=False, figsize=(8, 8), colors=colors)

# 4. Erstellung des Modells

Hier wird das Modell erstellt. Initial werden drei Parameter übergeben:

- der Modell-Modus (`training` oder `inference`)
- die oben definierte Konfiguration
- das Verzeichnis indem das Modell gespeichert / geladen werden soll

Anschließend werden die Gewichte des Modells initialisiert, mögliche Werte für `init_with` sind

- `imagenet`: initialisiert das Modell mit Gewichten die auf dem Imagenet Datensatz trainiert wurden
- `coco`: initialisiert das Modell mit Gewichten die auf dem MS COCO Datensatz trainiert wurden
- `last`: versucht das zuletzt trainierte Modell zu laden. Nur sinnvoll wenn ein solches Modell im angegebenen Ordner existiert
- `manual`: versucht das Modell aus der in `MANUAL_MODEL_PATH` angegebenen Datei zu laden

Aufgaben:

- Tragen Sie den passenden `mode` Parameter für den Modell-Modus ein
- Wählen Sie den passenden Wert für die `init_with` Variable um das Modell mit auf dem MS COCO Datensatz trainierten Gewichten zu initialisieren

In [ ]:
# Create model in training mode
model = modellib.MaskRCNN(mode="training", config=config,
                          model_dir=MODEL_DIR)

# Which weights to start with?
init_with = "___WERT_EINTRAGEN___"  # imagenet, coco, manual or last
MANUAL_MODEL_PATH = "/models/mask_rcnn_bone_age_0100.h5"

if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
    # Load weights trained on MS COCO, but skip layers that
    # are different due to the different number of classes
    # See README for instructions to download the COCO weights
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
    # Load the last model you trained and continue training
    model.load_weights(model.find_last(), by_name=True)
elif init_with == "manual":
    model.load_weights(MANUAL_MODEL_PATH, by_name=True)
    
print("Model loaded")

## 4.1 Training (Schritt 1)

Nun findet das Training des Modells statt.

Der Funktion [`model.train`](https://github.com/matterport/Mask_RCNN/blob/master/mrcnn/model.py#L2276) werden hierzu

- die Trainings- und Validierungsdatensätze,
- die Lernrate (die oben in der Konfiguration festgelegt wurde),
- die Anzahl Epochen (hier auf 1 gesetzt da es sonst zu lange dauert) und
- die zu trainierenden Schichten

übergeben.

Das Training geschieht in zwei Schritten:

1. nur die obersten, neu hinzugefügten Schichten. Dazu werden die Gewichte in den anderen Schichten eingefroren und beim Training nicht verändert. Um das zu erreichen, wird dem `layers` Parameter der Wert `heads` übergeben.

2. anschließendes Fine-tuning aller Schichten mit `layers="all"`

Abschließend kann das Modell bei Bedarf manuell gespeichert werden, dies ist jedoch üblicherweise nicht nötig, da das Modell automatisch nach jeder Epoche gespeichert wird.

Auf eine intelligente Vorverarbeitung der Bilder wurde an dieser Stelle verzichtet, dies ist jedoch in vielen Fällen ebenso wichtig oder noch wichtiger als eine gute Netzwerkachitektur und die Wahl geeigneter Lernparameter und würde in diesem Fall das Training deutlich leistungsstärkere Modelle in kürzerer Zeit erlauben.

Aufgaben:

- Wählen Sie die passenden Werte für die `layers`-Variablen

In [ ]:
history = LossHistory()

# Train the head branches
# Passing layers="heads" freezes all layers except the head
# layers. You can also pass a regular expression to select
# which layers to train by name pattern.
model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE, 
            epochs=2,
            custom_callbacks=[history],
            layers='___WERT_EINTRAGEN___')

print("Training finished")

Die Ausgabe des Trainingsscripts enthält folgende Informationen:

- `8/30`: Das aktuelle Minibatch. Wie oben beschrieben ergibt sich die Zahl 30 aus der Anzahl Trainingsbilder durch die Batch-Größe: `238 / 8 = 29.75`
- `413s 180ms/step`: Gesamtdauer für die Trainingsepoche sowie durchschnittliche Dauer eines Minibatches
- `ETA: 1:55`: Erwartete Restzeit
- `loss: 1.3854`: Wert der zu minimierenden Verlustfunktion auf dem letzten Minibatch
- `rpn_class_loss` und `rpn_bbox_loss`: Werte der Verlustfunktion für das Netzwerk welches die ROIs vorschlägt, einerseits für die Objektklassen und andererseits für die Bounding Boxes
- `mrcnn_class_loss`, `mrcnn_bbox_loss` und `mrcnn_mask_loss` sind die entsprechenden Werte der Verlustfunktion für die Klassifikation, die Bounding Boxen und die Segmentierungsmasken (M-RCNN arbeitet intern immer mit diesen Masken)
- `val_*:`: Die entsprechenden Werte auf dem gesamten Validierungsdatensatz. Wird nur einmal pro Epoche generiert

### 4.1.1 Visualisierung (Schritt 1)

In [ ]:
# summarize history for loss
plt.plot(*smooth(history.loss))
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('step')
plt.legend(['train'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(*smooth(history.rpn_class_loss))
plt.title('model RPN class loss')
plt.ylabel('loss')
plt.xlabel('step')
plt.legend(['train'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(*smooth(history.rpn_bbox_loss))
plt.title('model RPN bbox loss')
plt.ylabel('loss')
plt.xlabel('step')
plt.legend(['train'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(*smooth(history.mrcnn_class_loss))
plt.title('model MRCNN class loss')
plt.ylabel('loss')
plt.xlabel('step')
plt.legend(['train'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(*smooth(history.mrcnn_bbox_loss))
plt.title('model MRCNN bbox loss')
plt.ylabel('loss')
plt.xlabel('step')
plt.legend(['train'], loc='upper left')
plt.show()

### 4.1.2 Detektion (Schritt 1)

Nach dem Training schauen wir uns jetzt an wie gut unser Modell funktioniert. Dazu wird die Batch-Größe auf 1 gesetzt und ein Modell mit dem Modus `inference` erstellt und anschließend die Gewichte vom Training geladen.

In [ ]:
class InferenceConfig(BoneAgeConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()

# Recreate the model in inference mode
model_inference = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)

# Get path to saved weights
# Either set a specific path or find last trained weights
model_path = model_inference.find_last()

# Load trained weights
print("Loading weights from ", model_path)
model_inference.load_weights(model_path, by_name=True)

Hier wird nun ein je ein zufälliges Bild aus dem Trainings- und Validierungs-Datensatz genommen und zunächst das Original mit den zugehörigen ROIs und dann das Bild mit den vom Modell vorausgesagten ROIs und zugehörigen Konfidenzwerten angezeigt.

Um die Ausgabe auf die X "sichersten" ROIs für jede Klasse zu reduzieren kann die `filter_result`-Zeile einkommentiert werden.

Diese Zelle kann mehrfach ausgeführt werden.

In [ ]:
# Test on a random image
image_id = random.choice(dataset_train.image_ids)
original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset_train, inference_config, 
                           image_id )

colors = [color_palette[id] for id in gt_class_id]

print("Original (training):")
log("original_image", original_image)
log("image_meta", image_meta)
log("gt_class_id", gt_class_id)
log("gt_bbox", gt_bbox)
log("gt_mask", gt_mask)

visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id, 
                            dataset_train.class_names, show_mask=False, show_polygon=False, colors=colors, figsize=(8, 8))

results = model_inference.detect([original_image], verbose=1)
        
print("Prediction:")

r = results[0]
r = filter_result(r, {0: 0, 1: 5, 2: 5, 3: 5, 4: 1, 5: 1, 6: 1,})
print(r['rois'].shape)
print(r['masks'].shape)
print(r['class_ids'].shape)
print(r['scores'].shape)
colors = [color_palette[id] for id in r['class_ids']]
visualize.display_instances(original_image, r['rois'], r['masks'], r['class_ids'], 
                            dataset_val.class_names, r['scores'], ax=get_ax(), colors=colors, show_mask=False, show_polygon=False)

# Test on a random image
image_id = random.choice(dataset_val.image_ids)
original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset_val, inference_config, 
                           image_id)

colors = [color_palette[id] for id in gt_class_id]
plt.show()
print("\n\nOriginal (validation):")
log("original_image", original_image)
log("image_meta", image_meta)
log("gt_class_id", gt_class_id)
log("gt_bbox", gt_bbox)
log("gt_mask", gt_mask)

visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id, 
                            dataset_train.class_names, show_mask=False, show_polygon=False, colors=colors, figsize=(8, 8))

results = model_inference.detect([original_image], verbose=1)
        
print("Prediction:")

r = results[0]
r = filter_result(r, {0: 0, 1: 5, 2: 5, 3: 5, 4: 1, 5: 1, 6: 1,})
print(r['rois'].shape)
print(r['masks'].shape)
print(r['class_ids'].shape)
print(r['scores'].shape)
colors = [color_palette[id] for id in r['class_ids']]
visualize.display_instances(original_image, r['rois'], r['masks'], r['class_ids'], 
                            dataset_val.class_names, r['scores'], ax=get_ax(), colors=colors, show_mask=False, show_polygon=False)

## 4.2 Training (Schritt 2)

Nun wird das komplette Modell für eine Epoche mit reduzierter Lernrate trainiert.

Der `epochs`-Parameter besagt hier wie viele Epochen insgesamt trainiert werden soll. Da bereits für zwei Epochen trainiert wurde, wird durch die Angabe 4 hier nur noch zwei weitere Epoche trainiert.

In [ ]:
# Fine tune all layers
# Passing layers="all" trains all layers. You can also 
# pass a regular expression to select which layers to
# train by name pattern.
model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE / 10,
            epochs=4,
            custom_callbacks=[history],
            layers="all")

print("Training finished")

### 4.2.1 Visualisierung (Schritt 2)

In [ ]:
# summarize history for loss
plt.plot(*smooth(history.loss))
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('step')
plt.legend(['train'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(*smooth(history.rpn_class_loss))
plt.title('model RPN class loss')
plt.ylabel('loss')
plt.xlabel('step')
plt.legend(['train'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(*smooth(history.rpn_bbox_loss))
plt.title('model RPN bbox loss')
plt.ylabel('loss')
plt.xlabel('step')
plt.legend(['train'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(*smooth(history.mrcnn_class_loss))
plt.title('model MRCNN class loss')
plt.ylabel('loss')
plt.xlabel('step')
plt.legend(['train'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(*smooth(history.mrcnn_bbox_loss))
plt.title('model MRCNN bbox loss')
plt.ylabel('loss')
plt.xlabel('step')
plt.legend(['train'], loc='upper left')
plt.show()

### 4.2.2 Detektion (Schritt 2)

Hier erneut die Detektion nach dem vollständigen Training über 4 Epochen.

In [ ]:
class InferenceConfig(BoneAgeConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()

# Recreate the model in inference mode
model_inference = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)

# Get path to saved weights
# Either set a specific path or find last trained weights
#model_path = os.path.join(PRETRAINED_MODEL_DIR, "mask_rcnn_bone_age_0100.h5")
model_path = model_inference.find_last()

# Load trained weights
print("Loading weights from ", model_path)
model_inference.load_weights(model_path, by_name=True)

In [ ]:
# Test on a random image
image_id = random.choice(dataset_train.image_ids)
original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset_train, inference_config, 
                           image_id)

colors = [color_palette[id] for id in gt_class_id]

print("Original (training):")
log("original_image", original_image)
log("image_meta", image_meta)
log("gt_class_id", gt_class_id)
log("gt_bbox", gt_bbox)
log("gt_mask", gt_mask)

visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id, 
                            dataset_train.class_names, show_mask=False, show_polygon=False, colors=colors, figsize=(8, 8))

results = model_inference.detect([original_image], verbose=1)
        
print("Prediction:")

r = results[0]
r = filter_result(r, {0: 0, 1: 5, 2: 5, 3: 5, 4: 1, 5: 1, 6: 1,})
print(r['rois'].shape)
print(r['masks'].shape)
print(r['class_ids'].shape)
print(r['scores'].shape)
colors = [color_palette[id] for id in r['class_ids']]
visualize.display_instances(original_image, r['rois'], r['masks'], r['class_ids'], 
                            dataset_val.class_names, r['scores'], ax=get_ax(), colors=colors, show_mask=False, show_polygon=False)

# Test on a random image
image_id = random.choice(dataset_val.image_ids)
original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset_val, inference_config, 
                           image_id)

colors = [color_palette[id] for id in gt_class_id]
plt.show()
print("\n\nOriginal (validation):")
log("original_image", original_image)
log("image_meta", image_meta)
log("gt_class_id", gt_class_id)
log("gt_bbox", gt_bbox)
log("gt_mask", gt_mask)

visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id, 
                            dataset_train.class_names, show_mask=False, show_polygon=False, colors=colors, figsize=(8, 8))

results = model_inference.detect([original_image], verbose=1)
        
print("Prediction:")

r = results[0]
r = filter_result(r, {0: 0, 1: 5, 2: 5, 3: 5, 4: 1, 5: 1, 6: 1,})
print(r['rois'].shape)
print(r['masks'].shape)
print(r['class_ids'].shape)
print(r['scores'].shape)
colors = [color_palette[id] for id in r['class_ids']]
visualize.display_instances(original_image, r['rois'], r['masks'], r['class_ids'], 
                            dataset_val.class_names, r['scores'], ax=get_ax(), colors=colors, show_mask=False, show_polygon=False)

# 5. Pre-trained model

Da die Performance unseres Modells nach wenigen Epochen Training noch zu wünschen übrig lässt, laden wir hier ein Modell welches für 100 Epochen trainiert wurde und vergleichen die Performance.

In [ ]:
class InferenceConfig(BoneAgeConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()

# Recreate the model in inference mode
model_inference = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)

# Get path to saved weights
# Either set a specific path or find last trained weights
model_path = os.path.join(PRETRAINED_MODEL_DIR, "mask_rcnn_bone_age_0100.h5")

# Load trained weights
print("Loading weights from ", model_path)
model_inference.load_weights(model_path, by_name=True)
print("Model loaded")

Führen Sie nun selbst die Visualisierung durch.

# 6 Evaluation

Zur Evaluierung werden verschiedene Fehlermaße berechnet:

**Precision:** $P = \frac{TP}{TP + FP}$, wobei TP die _true positives_ (hier: es wurde eine Segmentierungsmaske zu einer Lesion gefunden) und FP die _false positives_ sind (hier: es wurde eine Segmentierungsmakse gefunden, wo tatsächlich keine Lesion ist), d.h. die Precision gibt an wie viele der gefundenen Segmentierungsmasken wirklich zu Lesions gehören.

**Recall:** $R = \frac{TP}{TP + FN}$, wobei FN die _false negatives_ sind (hier: Lesions zu denen keine Segmentierungsmasken gefunden wurden), d.h. Recall gibt an, zu wie vielen der tatsächlichen Lesions Segmentierungsmaksen gefunden wurden.

Bei der Bildklassifikation gibt es vorgegebene Kategorien über die einfach bestimmt werden kann, ob eine Dateninstanz korrekt klassifiziert wurde, doch wie sieht das bei der Erkennung mehrerer Objekte in einem aus? Wann gilt eine gefundene ROI als _true positive_?

## 6.1 IoU (Intersection over union)

Mit der IoU wird die Überlappung zweier Bildbereiche bestimmt:

<img src="https://miro.medium.com/max/1600/1*FrmKLxCtkokDC3Yr1wc70w.png" />
Quelle: https://medium.com/@jonathan_hui/map-mean-average-precision-for-object-detection-45c121a31173

Diese Zahl liegt zwischen 0 und 1, wobei 0 auf unseren Datensatz bezogen besagt dass die vorausgesagte ROI keine Überlappung mit einer tatsächlichen ROI dieser Klasse hat, und 1 besagt dass die ROI eine tatsächliche ROI der gleichen Klasse exakt abbildet.

Ob eine ROI nun für die oben beschriebenen Maße als korrekt angesehen wird hängt von der Wahl des IoU ab, im Folgenden wird mit einem IoU von 0.5 gearbeitet, d.h. sobald die Hälfte der gefundenen Segmentierungsmaske über einer tatsächlichen ROI liegt, wird sie als korrekt angesehen.

Der IoU-Grenzwert kann über die Variable `IOU_THRESHOLD` angepasst werden, die Anzahl der zu untersuchenden Bilder über die Variable `NUM_IMAGES`.

In der Funktion `print_condition` kann angepasst werden, wann ein untersuchtes Bild ausgegeben werden soll.

Ausgegeben werden nun folgende Werte:

- mean Average Precision (mAP) für das aktuelle Bild, dieser Wert berechnet sich aus der [Fläche unter der Precision-Recall-Kurve](https://medium.com/@jonathan_hui/map-mean-average-precision-for-object-detection-45c121a31173)
- IoU für jede der nummertierten, vorausgesagten ROIs
- Precision, Recall und die zugehörigen Variablen (Formel siehe oben)

In [ ]:
IOU_THRESHOLD = 0.5
NUM_IMAGES = 10

# change this if you want to print only certain images
def print_condition(AP, precisions, recalls, overlaps):
    # Print images below IoU threshold
    #return overlaps[0] < IOU_THRESHOLD
    # Uncomment to print all
    return True
    # Uncomment to print none
    #return False

# Compute VOC-Style mAP @ IoU=0.5
# Running on 10 images. Increase for better accuracy.
image_ids = np.random.choice(dataset_val.image_ids, NUM_IMAGES, replace=False)
APs = []
for image_id in image_ids:
    # Load image and ground truth data
    image, image_meta, gt_class_ids, gt_bbox, gt_masks =\
        modellib.load_image_gt(dataset_val, inference_config,
                               image_id)
    molded_images = np.expand_dims(modellib.mold_image(image, inference_config), 0)
    
    # Run object detection
    results = model_inference.detect([image], verbose=0)
    r = results[0]
    
    # Compute AP
    AP, precisions, recalls, overlaps =\
        utils.compute_ap(gt_bbox, gt_class_ids, gt_masks,
                         r["rois"], r["class_ids"], r["scores"], r['masks'],
                        iou_threshold=IOU_THRESHOLD)
    APs.append(AP)
    
    if print_condition(AP, precisions, recalls, overlaps):
        print("Original:")
        colors = [color_palette[id] for id in gt_class_id]
        visualize.display_instances(image, gt_bbox, gt_masks, gt_class_ids, 
                                    dataset_train.class_names, figsize=(8, 8),
                                    colors=colors, show_polygon=False, show_mask=False)
        print("Prediction:")
        print("mAP:", AP)
        ious_raw = list(map(lambda x: max(x), overlaps))
        ious = list(map(lambda x, n: str(n) + ": " + str(x), ious_raw, list(range(1, len(r['class_ids'])+1))))
        print("IoU:", ious)
        
        true_positives = len(list(filter(lambda x: x > IOU_THRESHOLD, ious_raw)))
        false_positives = len(ious_raw) - true_positives
        false_negatives = len(gt_class_ids) - true_positives
        print("True positives (predicted objects where overlap >=", IOU_THRESHOLD, "):", true_positives)
        print("False positives (predicted objects where overlap <", IOU_THRESHOLD, "):", false_positives)
        print("False negatives (ground truth objects for which there was no predicted object with overlap >=", IOU_THRESHOLD, "):", false_negatives)
        
        print("Precision: ", true_positives, " / (", true_positives, " + ", false_positives, ") = ", precisions[-2])
        print("Recall: ", true_positives, " / (", true_positives, " + ", false_negatives, ") = ", recalls[-2])
        colors = [color_palette[id] for id in r['class_ids']]
        visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                                    list(range(len(r['masks']))), r['scores'], ax=get_ax(),
                                    colors=colors, show_polygon=False, show_mask=False,
                                   captions=list(range(1, len(r['class_ids'])+1)))
        plt.show()
    
print("Overall mAP: ", np.mean(APs))